In [1]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize as op
import math
import torch
import re

In [2]:
data = pandas.read_csv('./data/Vout_sigmoid.csv')
data_split = data.iloc[:, 1:]
data_split

,"/Vout (R1_data=650000,R2_data=50000,W1_data=0.00021,L1_data=0.00011,W2_data=0.000725,L2_data=7e-05) Y","/Vout (R1_data=450000,R2_data=70000,W1_data=0.00047,L1_data=0.00017,W2_data=0.000575,L2_data=5e-05) Y","/Vout (R1_data=500000,R2_data=55000,W1_data=0.000405,L1_data=0.000185,W2_data=0.000612,L2_data=4.5e-05) Y","/Vout (R1_data=700000,R2_data=75000,W1_data=0.000145,L1_data=0.000125,W2_data=0.000762,L2_data=6.5e-05) Y","/Vout (R1_data=600000,R2_data=45000,W1_data=0.000535,L1_data=0.000155,W2_data=0.000688,L2_data=7.5e-05) Y","/Vout (R1_data=400000,R2_data=65000,W1_data=0.000275,L1_data=9.5e-05,W2_data=0.000538,L2_data=5.5e-05) Y","/Vout (R1_data=425000,R2_data=52500,W1_data=0.000568,L1_data=0.000132,W2_data=0.000669,L2_data=5.2e-05) Y","/Vout (R1_data=625000,R2_data=72500,W1_data=0.000308,L1_data=0.000192,W2_data=0.000519,L2_data=7.2e-05) Y","/Vout (R1_data=725000,R2_data=42500,W1_data=0.000438,L1_data=0.000102,W2_data=0.000594,L2_data=6.2e-05) Y","/Vout (R1_data=525000,R2_data=62500,W1_data=0.000178,L1_data=0.000162,W2_data=0.000744,L2_data=4.2e-05) Y",...,"/Vout (R1_data=401831,R2_data=69358,W1_data=0.000104,L1_data=9.8e-05,W2_data=0.000698,L2_data=5.8e-05) Y","/Vout (R1_data=426831,R2_data=51858,W1_data=0.000396,L1_data=0.000136,W2_data=0.000529,L2_data=5e-05) Y","/Vout (R1_data=626831,R2_data=71858,W1_data=0.000136,L1_data=0.000196,W2_data=0.000679,L2_data=7e-05) Y","/Vout (R1_data=726831,R2_data=41858,W1_data=0.000526,L1_data=0.000106,W2_data=0.000754,L2_data=6e-05) Y","/Vout (R1_data=526831,R2_data=61858,W1_data=0.000266,L1_data=0.000166,W2_data=0.000604,L2_data=4e-05) Y","/Vout (R1_data=476831,R2_data=46858,W1_data=0.000201,L1_data=0.000151,W2_data=0.000642,L2_data=5.5e-05) Y","/Vout (R1_data=676831,R2_data=66858,W1_data=0.000461,L1_data=9.1e-05,W2_data=0.000792,L2_data=7.5e-05) Y","/Vout (R1_data=576831,R2_data=56858,W1_data=0.000331,L1_data=0.000181,W2_data=0.000717,L2_data=6.5e-05) Y","/Vout (R1_data=376831,R2_data=76858,W1_data=0.000591,L1_data=0.000121,W2_data=0.000567,L2_data=4.5e-05) Y","/Vout (R1_data=550000,R2_data=60000,W1_data=0.00034,L1_data=0.00014,W2_data=0.00065,L2_data=6e-05) Y"
0,0.05294,0.03398,0.03665,0.03193,0.06537,0.04246,0.04016,0.05235,0.06703,0.0251,...,0.03265,0.04904,0.03971,0.05253,0.02962,0.04942,0.0394,0.04375,0.0282,0.04217
1,0.05294,0.03398,0.03665,0.03193,0.06537,0.04246,0.04016,0.05235,0.06703,0.0251,...,0.03265,0.04904,0.03971,0.05253,0.02962,0.04942,0.0394,0.04375,0.0282,0.04217
2,0.05294,0.03398,0.03665,0.03193,0.06537,0.04246,0.04016,0.05235,0.06703,0.0251,...,0.03265,0.04904,0.03971,0.05253,0.02962,0.04942,0.0394,0.04375,0.0282,0.04217
3,0.05294,0.03398,0.03665,0.03193,0.06537,0.04246,0.04016,0.05235,0.06703,0.0251,...,0.03265,0.04904,0.03971,0.05253,0.02962,0.04942,0.0394,0.04375,0.0282,0.04217
4,0.05294,0.03398,0.03665,0.03193,0.06537,0.04246,0.04016,0.05235,0.06703,0.0251,...,0.03265,0.04904,0.03971,0.05253,0.02962,0.04942,0.0394,0.04375,0.0282,0.04217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,0.99980,0.99980,0.99980,0.99970,0.99990,0.99980,0.99980,0.99980,0.99990,0.9997,...,0.99970,0.99980,0.99980,0.99980,0.99970,0.99980,0.9998,0.99980,0.9997,0.99980
397,0.99980,0.99980,0.99980,0.99970,0.99990,0.99980,0.99980,0.99980,0.99990,0.9997,...,0.99970,0.99980,0.99980,0.99980,0.99970,0.99980,0.9998,0.99980,0.9997,0.99980
398,0.99980,0.99980,0.99980,0.99970,0.99990,0.99980,0.99980,0.99980,0.99990,0.9997,...,0.99970,0.99980,0.99980,0.99980,0.99970,0.99980,0.9998,0.99980,0.9997,0.99980
399,0.99980,0.99980,0.99980,0.99970,0.99990,0.99980,0.99980,0.99980,0.99990,0.9997,...,0.99970,0.99980,0.99980,0.99980,0.99970,0.99980,0.9998,0.99980,0.9997,0.99980


In [4]:
datalist=data_split.columns.values.tolist()
print(datalist)
print(len(datalist))

['/Vout (R1_data=650000,R2_data=50000,W1_data=0.00021,L1_data=0.00011,W2_data=0.000725,L2_data=7e-05) Y', '/Vout (R1_data=450000,R2_data=70000,W1_data=0.00047,L1_data=0.00017,W2_data=0.000575,L2_data=5e-05) Y', '/Vout (R1_data=500000,R2_data=55000,W1_data=0.000405,L1_data=0.000185,W2_data=0.000612,L2_data=4.5e-05) Y', '/Vout (R1_data=700000,R2_data=75000,W1_data=0.000145,L1_data=0.000125,W2_data=0.000762,L2_data=6.5e-05) Y', '/Vout (R1_data=600000,R2_data=45000,W1_data=0.000535,L1_data=0.000155,W2_data=0.000688,L2_data=7.5e-05) Y', '/Vout (R1_data=400000,R2_data=65000,W1_data=0.000275,L1_data=9.5e-05,W2_data=0.000538,L2_data=5.5e-05) Y', '/Vout (R1_data=425000,R2_data=52500,W1_data=0.000568,L1_data=0.000132,W2_data=0.000669,L2_data=5.2e-05) Y', '/Vout (R1_data=625000,R2_data=72500,W1_data=0.000308,L1_data=0.000192,W2_data=0.000519,L2_data=7.2e-05) Y', '/Vout (R1_data=725000,R2_data=42500,W1_data=0.000438,L1_data=0.000102,W2_data=0.000594,L2_data=6.2e-05) Y', '/Vout (R1_data=525000,R2_d

In [5]:
def extract_values(data_list):
    patterns = [
        r"R1_data=(-?\d+.?\d*(?:e-?\d+)?)(?=,|\))",
        r"R2_data=(-?\d+.?\d*(?:e-?\d+)?)(?=,|\))",
        r"W1_data=(-?\d+.?\d*(?:e-?\d+)?)(?=,|\))",
        r"L1_data=(-?\d+.?\d*(?:e-?\d+)?)(?=,|\))",
        r"W2_data=(-?\d+.?\d*(?:e-?\d+)?)(?=,|\))",
        r"L2_data=(-?\d+.?\d*(?:e-?\d+)?)(?=,|\))"
    ]
    
    matrix = []
    
    for data in data_list:
        row = []
        for pattern in patterns:
            match = re.search(pattern, data)
            if match:
                value = match.group(1)
                if 'e' in value or '.' in value: 
                    row.append(float(value))
                else:
                    row.append(int(value))
            else:
                row.append(None)
        matrix.append(row)
    return matrix

result = extract_values(datalist)

In [6]:
result_temp = torch.tensor(result)
result_temp.max(0)[0]

tensor([7.4995e+05, 7.9995e+04, 6.0000e-04, 2.0000e-04, 8.0000e-04, 8.0000e-05])

In [7]:
result_temp.min(0)[0]

tensor([3.5005e+05, 4.0002e+04, 8.0000e-05, 8.0000e-05, 5.0000e-04, 4.0000e-05])

In [8]:
torch.save(result_temp, './data/sigmoid_RT.data')

In [9]:
r1 = result_temp[:, 0].view(-1,1)
r2 = result_temp[:, 1].view(-1,1)
w1 = result_temp[:, 2].view(-1,1)
l1 = result_temp[:, 3].view(-1,1)
w2 = result_temp[:, 4].view(-1,1)
l2 = result_temp[:, 5].view(-1,1)
ratio_1 = w1/l1
ratio_2 = w2/l2

r1.shape, r2.shape, w1.shape, l1.shape, w2.shape, l2.shape, ratio_1.shape, ratio_2.shape

(torch.Size([9999, 1]),
 torch.Size([9999, 1]),
 torch.Size([9999, 1]),
 torch.Size([9999, 1]),
 torch.Size([9999, 1]),
 torch.Size([9999, 1]),
 torch.Size([9999, 1]),
 torch.Size([9999, 1]))

In [10]:
C = torch.cat([r1, r2, w1, l1, w2, l2, ratio_1, ratio_2], dim=1)
C.shape

torch.Size([9999, 8])

In [11]:
torch.save(C, './data/sigmoid_RT_ratio.data')